## Задание

Реализуйте поиск по [Quora question pairs](https://www.kaggle.com/loopdigga/quora-question-pairs-russian) на нескольких векторных моделях

    1. fasttext, модель ruscorpora_none_fasttextskipgram_300_2_2019
    2. elmo, модель ruwikiruscorpora_lemmas_elmo_1024_2019
    3. bert*, RuBERT - необязательно
   
Первые две обученные модели можно скачать на сайте [rusvectores](https://rusvectores.org/en/models/).

BERT делать необязательно, но если сделаете, 6 за курс у вас автоматом. Модель можно [найти тут](http://docs.deeppavlov.ai/en/master/features/models/bert.html).

In [2]:
pip install gensim

Note: you may need to restart the kernel to use updated packages.


In [3]:
%load_ext autoreload

from gensim.models import Word2Vec, KeyedVectors
from gensim.models import FastText
from gensim.test.utils import get_tmpfile

## word2vec + fasttext

получение вектора документа

In [4]:
model_file = 'model.model'

In [5]:
model_file

'model.model'

In [6]:
word_vectors = KeyedVectors.load(model_file)

In [7]:
'алмаз' in word_vectors.vocab

True

In [8]:
vector = word_vectors['алмаз']  # numpy vector of a word
vector.shape

(300,)

In [9]:
import os
os.chdir('C:/Users/home/sem3_infosearch')

In [10]:
import pandas as pd

In [11]:
table = pd.read_csv('quora_question_pairs_rus.csv')

In [12]:
table.head()

,Unnamed: 0,question1,question2,is_duplicate
0,0,Какова история кохинор кох-и-ноор-бриллиант,"что произойдет, если правительство Индии украд...",0
1,1,как я могу увеличить скорость моего интернет-с...,как повысить скорость интернета путем взлома ч...,0
2,2,"почему я мысленно очень одинок, как я могу это...","найти остаток, когда математика 23 ^ 24 матема...",0
3,3,которые растворяют в воде быстро сахарную соль...,какая рыба выживет в соленой воде,0
4,4,астрология: я - луна-колпачок из козерога и кр...,Я тройная луна-козерог и восхождение в козерог...,1


In [13]:
pip install pymorphy2

Note: you may need to restart the kernel to use updated packages.


In [14]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [15]:
texts = []
for row in table['question2']:
    texts.append(str(row))

In [16]:
import re
import nltk
nltk.download('stopwords')
stemmer = nltk.stem.PorterStemmer()
stopwords = set(nltk.corpus.stopwords.words('russian'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\home\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [18]:
#tokenize documents
# сделали препроцессинг, получили леммы 
allWordsAllTexts = []
for text in texts[0:10000]:
    allWordsInOneText = []
    if text != "":
        for word in text.split():
            if word not in stopwords:
                word = re.sub('[.,-;:?!@#$%^&()_+=—\ufeff–"…«»>]', '', word).lower()
                if word != "":
                    p = morph.parse(word)[0]
                    if p not in allWordsInOneText:
                        allWordsInOneText.append(p.normal_form)
        allWordsAllTexts.append(allWordsInOneText)

In [19]:
allWordsAllTexts[:2]

[['произойти',
  'правительство',
  'индия',
  'украсть',
  'кохинор',
  'кохинооралмаз',
  'назад'],
 ['повысить', 'скорость', 'интернет', 'путём', 'взлом', 'dns']]

In [20]:
import numpy as np

In [21]:
vectors = []
for s in allWordsAllTexts:
    # создаем маски для векторов 
    lemmas_vectors = np.zeros((len(s), word_vectors.vector_size))
    vec = np.zeros((word_vectors.vector_size,))
    # если слово есть в модели, берем его вектор
    for idx, lemma in enumerate(s):
        if lemma in word_vectors.vocab:
            lemmas_vectors[idx] = word_vectors[lemma]
            if lemmas_vectors.shape[0] is not 0:
                vec1 = np.mean(lemmas_vectors, axis=0)
    vectors.append(vec1)

In [22]:
inputs = []
for row in table['question1']:
    inputs.append(str(row))

In [23]:
#tokenize inputs
# сделали препроцессинг, получили леммы 
allWordsAllInputs = []
for text in inputs[0:10]:
    allWordsInOneInput = []
    if text != "":
        for word in text.split():
            if word not in stopwords:
                word = re.sub('[.,-;:?!@#$%^&()_+=—\ufeff–"…«»>]', '', word).lower()
                if word != "":
                    p = morph.parse(word)[0]
                    if p not in allWordsInOneInput:
                        allWordsInOneInput.append(p.normal_form)
        allWordsAllInputs.append(allWordsInOneInput)

In [24]:
vectors_inputs = []
for s in allWordsAllInputs:
    # создаем маски для векторов 
    lemmas_vectors_inputs = np.zeros((len(s), word_vectors.vector_size))
    # если слово есть в модели, берем его вектор
    for idx, lemma in enumerate(s):
        if lemma in word_vectors.vocab:
            lemmas_vectors_inputs[idx] = word_vectors[lemma]
            if lemmas_vectors_inputs.shape[0] is not 0:
                vec = np.mean(lemmas_vectors_inputs, axis=0)
    vectors_inputs.append(vec)

In [25]:
import sklearn
from sklearn.metrics.pairwise import cosine_similarity

### __Задача 1__:    
Сравните время индексации корпуса для каждой модели 

In [27]:
#создаём словарь: "индекс запроса:индекс наиболее релевантного текста"
#время индексациии для модели fasttext на 9 запросах и 10000 документах
import time
start_time = time.time()
bests = {}
for idx, i in enumerate(vectors_inputs):
    m = []
    mean_input = i.reshape(1, -1)
    for idx1, j in enumerate(vectors):
        mean_text = j.reshape(1, -1)
        sk = cosine_similarity(mean_input, mean_text, dense_output=True)
        m.append(sk[0][0])
    best_result = m.index(max(m))
    bests[idx] = best_result
print("--- %s seconds ---" % (time.time() - start_time))

--- 67.29096245765686 seconds ---


In [28]:
print(bests)

{0: 8236, 1: 3418, 2: 3961, 3: 6108, 4: 4, 5: 9199, 6: 6, 7: 7, 8: 1742, 9: 2940}


In [37]:
#пример того, как работает поисковик
print("'" + table['question1'][1] + "'" + ' - ' + "'" + table['question2'][3418] + "'")

'как я могу увеличить скорость моего интернет-соединения, используя vpn' - 'как я могу увеличить свой вес'


In [36]:
print("'" + table['question1'][2] + "'" + ' - ' + "'" + table['question2'][3961] + "'")

'почему я мысленно очень одинок, как я могу это решить' - 'у меня есть этот страх во мне, что заставляет меня ненавидеть разговаривать с людьми, особенно с девушками, которых я боюсь, когда я не могу придумать ничего, чтобы сказать, и это превращается в неловкий момент, к которому разговор впоследствии становится скучным, я не хочу быть скучный человек, как я могу исправить это'


### __Задача 2__:    
Выведите качество поиска для каждой модели +  BM25 для сравнения

Качество оцениваем так же, как в прошлом задании:
    - если в топ-5 результатов выдачи попал хоть один релевантный документ, выдача точная
    - если в топ-5 нет ни одного релеватного документа, выдача получает 0
   